In [ ]:
# Install pyfuzzy-toolbox with machine learning module
!pip install pyfuzzy-toolbox[ml] -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import fuzzy_systems as fs
from fuzzy_systems import MamdaniSystem
from fuzzy_systems.learning import MamdaniLearning

%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

In [ ]:
# Target function
def true_function(x):
    """f(x) = -2x + 5"""
    return -2 * x + 5

# Generate training data
n_samples = 100
X_train = np.linspace(0, 1, n_samples).reshape(-1, 1)
y_train = true_function(X_train.flatten()) + np.random.normal(0, 0.01, n_samples)

X_test = np.linspace(0, 1, 200).reshape(-1, 1)
y_true = true_function(X_test.flatten())

# Visualize
plt.figure(figsize=(10, 6))
plt.plot(X_test, y_true, 'k-', linewidth=3, label='f(x) = -2x + 5', alpha=0.7)
plt.scatter(X_train, y_train, color='red', s=30, alpha=0.5, label=f'Data (n={n_samples})')
plt.xlabel('x', fontsize=13)
plt.ylabel('y', fontsize=13)
plt.title('Linear Fit by Mamdani FIS', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# Create Fuzzy System
# ============================================================================

fis = MamdaniSystem()
input_universe = X_train.min(), X_train.max()
diff = y_train.max() - y_train.min()

output_universe = (y_train.min() - 0.1*diff, y_train.max() + 0.1*diff)

fis.add_input('x', input_universe)
fis.add_output('y', output_universe)

num_input_mfs = 7
num_output_mfs = 7

input_centers = np.linspace(input_universe[0], input_universe[1], num_input_mfs)
del_in = (input_centers[1] - input_centers[0])
for i in range(num_input_mfs):
    a = input_centers[i] - del_in
    b = input_centers[i]
    c = input_centers[i] + del_in
    fis.add_term('x', f'IN_{i+1}', 'trimf', (a, b, c))     

output_centers = np.linspace(output_universe[0], output_universe[1], num_output_mfs)
del_out = (output_centers[1] - output_centers[0])
for i in range(num_output_mfs):
    a = output_centers[i] - del_out
    b = output_centers[i]
    c = output_centers[i] + del_out
    fis.add_term('y', f'OUT_{i+1}', 'trimf', (a, b, c))

fis.info()

In [ ]:
# Instantiate the learning model
learner = MamdaniLearning(fis, num_points=1000, verbose=True)

In [ ]:
# Train fuzzy rules using Simulated Annealing optimizer
learner.fit_rules(
    X_train, y_train,
    optimizer='sa',
    optimizer_params={
        'temperature_init': 100.0, 
        'cooling_rate': 0.95, 
        'max_iterations': 5000,
        'temperature_min': 1e-6
    },
    initial_solution_method='random'
)

y_pred_train = learner.predict(X_train)
y_pred = learner.predict(X_test)

rmse_train = learner.get_cost()
rmse_test = learner.score(X_test, y_true)

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

history = learner.get_history()

# 1. Approximation
axes[0, 0].plot(X_test, y_true, 'b-', linewidth=3, label='Real', alpha=0.7)
axes[0, 0].plot(X_test, y_pred, 'r--', linewidth=2.5, label='FIS', alpha=0.8)
axes[0, 0].scatter(X_train, y_train, alpha=0.3, s=20, c='blue')
axes[0, 0].set_xlabel('x', fontsize=12)
axes[0, 0].set_ylabel('y', fontsize=12)
axes[0, 0].set_title(f'Approximation\nRMSE = {rmse_test:.4f}', 
                     fontsize=13, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# 2. Error
error = y_pred - y_true
axes[0, 1].plot(X_test, error, 'g-', linewidth=2.5)
axes[0, 1].axhline(y=0, color='k', linestyle='--')
axes[0, 1].fill_between(X_test.flatten(), 0, error, alpha=0.3, color='green')
axes[0, 1].set_xlabel('x', fontsize=12)
axes[0, 1].set_ylabel('Error', fontsize=12)
axes[0, 1].grid(True, alpha=0.3)

# 3. Convergence
axes[1, 0].plot(history['current_cost'], linewidth=2.5, color='darkblue')
axes[1, 0].set_xlabel('Iteration', fontsize=12)
axes[1, 0].set_ylabel('Fitness (RMSE)', fontsize=12)
axes[1, 0].set_title('FIS Convergence', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Error Distribution
axes[1, 1].hist(error, bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Error', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title('Error Distribution', fontsize=13, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize learned rule matrix
fig,ax = fis.plot_rule_matrix(figsize=(15,8))
fig.axes[0].set_title('Rule Visualization', fontsize=14, fontweight='bold', pad=40)
fig.axes[1].remove()
ax.images[0].set_cmap('ocean')


In [ ]:
# ============================================================================
# Compare Different Optimizers
# ============================================================================

# Define optimizer configurations
optimizers_config = {
    'SA': {
        'optimizer': 'sa',
        'params': {
            'temperature_init': 100.0,
            'cooling_rate': 0.95,
            'max_iterations': 2000,
            'plateau_iterations': 500,
            'temperature_min': 1e-6
        }
    },
    'GA': {
        'optimizer': 'ga',
        'params': {
            'pop_size': 50,
            'max_gen': 100,
            'mutation_rate': 0.1,
            'crossover_rate': 0.8
        }
    },
    'PSO': {
        'optimizer': 'pso',
        'params': {
            'n_particles': 30,
            'n_iterations': 100,
            'w_max': 0.9,
            'w_min': 0.4
        }
    },
    'DE': {
        'optimizer': 'de',
        'params': {
            'pop_size': 30,
            'max_iter': 100,
            'F': 0.8,
            'CR': 0.7
        }
    }
}

# Train with selected optimizer (default: GA)
learner = MamdaniLearning(fis, num_points=1000, verbose=True)
selected_optimizer = 'GA'
learner.fit_rules(
    X_train, y_train,
    optimizer=optimizers_config[selected_optimizer]['optimizer'],
    optimizer_params=optimizers_config[selected_optimizer]['params'],
    initial_solution_method='random'
)

In [ ]:
# Training complete
# Use the trained learner for predictions
y_pred_final = learner.predict(X_test)
rmse_final = learner.score(X_test, y_true)
print(f"Final RMSE: {rmse_final:.4f}")